In [1]:
import time
import struct

import chipwhisperer as cw

In [2]:
def as_hex(s):
    return "".join([f"\\x{ord(c):02x}" for c in s])

def make_pkt(data):
    pkt = b"\x01"
    pkt += struct.pack('!H', len(data))
    pkt += data

    # Checksum
    chk = (256 - sum(pkt[1:])) & 0xff
    pkt += bytes([chk, 0x03])

    return pkt

In [3]:
scope = cw.scope()

In [4]:
scope.default_setup()
scope.io.tio1 = 'serial_tx'
scope.io.tio2 = 'serial_rx'
scope.io.tio3 = 'gpio_low'
scope.io.glitch_trig_mcx = 'glitch'

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.clock.clkgen_freq                  changed from 0                         to 7370129.87012987         
scope.clock.adc_freq                     changed from 0                         to 29480519.48051948        
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2                            changed from serial_rx                 to serial_tx                
scope.io.hs2                             changed from None                      to clkgen                   
scope.io.tio_states

In [6]:
# scope.clock.adc_mul = 0
# scope.clock.clkgen_src = 'system'
# scope.clock.clkgen_freq = 200e6


scope.clock.clkgen_src = 'system'
scope.clock.clkgen_freq = 100e6
scope.clock.adc_mul = 0

target = cw.target(scope)
target.baud = 9600
target.flush()

scope.trigger.triggers = 'tio1'
scope.trigger.module = 'UART'
scope.gain.db = 12

scope.UARTTrigger.enabled = True
scope.UARTTrigger.baud = 9600
scope.UARTTrigger.set_pattern_match(0, '\x22\x03\x01\x00\x01\x00\xff\x03') # Synchronize
scope.UARTTrigger.trigger_source = 0

scope.glitch.enabled = True
scope.glitch.clk_src = "pll"

scope.io.glitch_hp = False
scope.io.glitch_lp = True
scope.io.glitch_trig_mcx = 'glitch'

scope.glitch.output = "enable_only"
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called

In [7]:
def read_block(target, block):
    start, end = block

    print("reading flash", hex(start), hex(end))
    pkt = make_pkt(b'\x15' + struct.pack('>II', start, end))

    target.write(pkt)   
    resp = target.read(6)
    if resp != '\x81\x00\x01\x15\xea\x03':
        print("wrong ack", as_hex(resp))
        return None

    rem = end - start
    dat = b""
    
    while rem > 0:
        target.write(b'\x81\x00\x01\x15\xea\x03')
        length = target.read(3)
        
        if len(length) != 3:
            print("wrong length received", as_hex(length))
            return None
        
        length = bytes([ord(c) for c in length])
        length = struct.unpack('!H', length[1:])[0] - 1 # Data starts with 0x15

        resp = b""
        while len(resp) < length + 3: 
            resp += bytes([ord(c) for c in target.read(target.in_waiting())])
            time.sleep(0.05)

        dat += resp[1:-2]
        rem -= length
    return dat

In [14]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["ext_offset", "repeat"])
min_delay = int(8.5e-6 * scope.clock.clkgen_freq)
max_delay = int(9.5e-6 * scope.clock.clkgen_freq)

print(min_delay, max_delay)

gc.set_range("ext_offset", min_delay, max_delay)
gc.set_range("repeat", 5, 20) 
gc.set_global_step(1)

gc.display_stats()
gc.glitch_plot(plotdots={"success":"+r", "reset":"xy", "normal":'xg'})


850 950


IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=850.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=950.0,…

FloatSlider(value=5.0, continuous_update=False, description='repeat setting:', disabled=True, max=20.0, min=5.…

:DynamicMap   []
   :Overlay
      .Points.I   :Points   [ext_offset,repeat]
      .Points.II  :Points   [ext_offset,repeat]
      .Points.III :Points   [ext_offset,repeat]

In [15]:
blocks = [(0x0, 0x1fff)]

while True:
    for delay, width in gc.glitch_values(clear=False):
        scope.glitch.ext_offset = delay
        scope.glitch.repeat = width
        scope.glitch.width = 0
        scope.glitch.offset = 0

        scope.arm()

        scope.io.tio3 = 'gpio_high' # Release Reset
        time.sleep(.2)
        target.read(target.in_waiting()) # Clean all bytes in serial input buffer

        try:
            # Sync
            target.write(b'\x00' * 10)
            assert target.read(1) == '\x00'

            # Connect
            target.write(b'\x55')
            assert target.read(1) == '\xc1'

            # Get device type
            target.write(b'\x01\x00\x01\x38\xc7\x03')
            assert target.read(6) == '\x81\x00\x01\x38\xc7\x03', "Wrong Resp Device Type 1"

            target.write(b'\x81\x00\x01\x38\xc7\x03')
            resp = target.read(30)
            if resp != '\x81\x00\x19\x38\x10\x01\xff\x40\x04\x28\x2c\x00\x01\x6e\x36\x00\x00\x7a\x12\x00\x07\x27\x0e\x00\x00\xf4\x24\x00\x82\x03':
                print("unexpected resp device type 2", as_hex(resp))

            # Set up frequencies
            target.write(make_pkt(b'\x32\x00\xf4\x24\x00\x07\x27\x0e\x00'))

            resp = target.read(6)
            if resp != '\x81\x00\x01\x32\xcd\x03':
                print("wrong resp freq 1", as_hex(resp))  

            target.write(b'\x81\x00\x01\x32\xcd\x03')
            resp = target.read(14)
            if resp != '\x81\x00\x09\x32\x07\x27\x0e\x00\x01\xc9\xc3\x80\x7c\x03':
                print("unexpected resp freq 2", as_hex(resp.hex))

            # Set bitrate (9600 baud)
            target.write(b'\x01\x00\x05\x34\x00\x00\x25\x80\x22\x03')
            resp = target.read(6)
            if resp != '\x81\x00\x01\x34\xcb\x03':
                print("wrong resp bitrate 1", as_hex(resp))

            # Synchronize!
            target.write(b'\x01\x00\x01\x00\xff\x03')
            resp = target.read(7)


            if resp == '\x81\x00\x01\x00\xff\x03':
                print(delay, width, "ok")
                resp = read_block(target, blocks[0])  
                if resp is not None:
                    print("Got", len(resp))
                    #raise RuntimeError("DONE!")
                    gc.add("success")
                else:
                    gc.add("reset")
            elif resp == '\x81\x00\x02\x80\xdc\xa2\x03': # Programmer Disabled
                gc.add("normal")
                continue
            elif resp == '\x81\x00\x02\x80\xc1\xbd\x03': # Packet error
                gc.add("reset")
                continue    
            elif resp == '\x81\x00\x02\x80\xc3\xbb\x03': # Flow error
                gc.add("reset")
                continue   
            elif resp == '':
                gc.add("reset")
                continue
            else:
                # print(delay, width, "wrong resp synchronize", as_hex(resp))
                gc.add("reset")
                continue


        except AssertionError as e:
            print(delay, width, e)
            gc.add("reset")

        finally:
            scope.io.tio3 = 'gpio_low' # Assert reset
            time.sleep(.1)



877 12 ok
reading flash 0x0 0x1fff
wrong ack 
923 15 ok
reading flash 0x0 0x1fff
Got 8192
936 13 ok
reading flash 0x0 0x1fff
Got 8192


KeyboardInterrupt: 